# 第十九讲：

## 强化学习算法的调试

我们在[第十一讲](chapter11.ipynb)中，在关于机器学习算法的调试介绍过直升机模型提到：

1. 搭建一个遥控直升机模拟器，主要是对状态转换概率$P_{sa}$进行建模（比如通过学习物理知识，依照空气动力学原理编写模拟器；或者也可以收集大量试验数据，通过拟合这些数据一个线性或非线性模型，以得到一个用当前状态、当前操作表示下一步状态的函数）；
2. 选择奖励函数，比如$R(s)=\left\lVert s-s_\mathrm{desired}\right\rVert^2$（$s$代表直升机当前的位置，这里示例的函数表示直升机实际位置与期望位置的平方误差，我们在[上一讲](chapter18.ipynb)的LQR中已经使用过这个奖励函数了）；
3. 在模拟器中运行强化学习算法控制直升机并尝试最大化奖励函数，$\mathrm E\left[R(s_0)+R(s_1)+\cdots+R(s_T)\right]$，进而求得对应的策略$\pi_\mathrm{RL}$。

现在，假设我们已经按照上述步骤得到了控制模型，但是发现该模型比人类直接操作差很多，接下来应该怎么办呢？我们可能会：
* 改进模拟器（也许模拟器所用的模型不是线性或非线性的；也许需要收集更多数据来拟合模型；也许需要调整拟合模型时所使用的特征值）；
* 修改奖励函数$R$（也许它不只是一个二次函数）；
* 改进学习算法（也许RL并不适合这个问题；也许需要对状态进行更精细的描述；也许在价值函数近似过程中使用的特征值需要调整）；

对于调整算法这样的任务，我们有太多的选择方向，如果选择错误，则将会浪费大量的时间精力。

接下来我们假设：
1. 直升机模拟器足够精确；
2. RL算法能够在模拟器中正确的控制飞机，因此算法正确的最大化了预期总收益$V_t^{\pi_\mathrm{RL}}(s_0)=\mathrm E\left[R(s_0)+R(s_1)+\cdots+R(s_T)\mid\pi_\mathrm{RL},s_t=s_0\right]$；
3. 最大化预期总收益与模型正确的操作飞机强相关。

如果上述三条假设均正确，则可以推出$\pi_\mathrm{RL}$能够控制好真正的直升机。

于是，我们可以**依次**做出如下诊断：
1. 首先，如果$\pi_\mathrm{RL}$在模拟器中飞的很好，但是在实际飞机上却表现不好，则说明是模拟器的问题；
2. 然后，用$\pi_\mathrm{human}$表示人类操纵者在操纵直升机时使用的策略，比较人类和算法的策略所对应的价值函数，如果$V^{\pi_\mathrm{RL}}\lt V^{\pi_\mathrm{human}}$，则说明是RL算法的问题。（算法并没有成功的最大化预期总收益。）
3. 最后，如果经比较发现$V^{\pi_\mathrm{RL}}\gt V^{\pi_\mathrm{human}}$，则说明问题出在奖励函数上。（最大化这个奖励函数并没有使得飞行水平提高。）

以上仅是一个强化学习调试的例子，因为我们恰好找到了一个极优秀的直升机操纵者，如果没有的话则需要想出别的调试方法。在通常的问题中，我们都需要自己找出针对问题的有效的调试方法。

在继续学习之前，我们先来回顾一下[上一讲](chapter18.ipynb)的内容：
* 其目标是$\displaystyle\max_a\mathrm E\left[R^{(0)}(s_0,a_0)+R^{(1)}(s_1,a_1)+\cdots+R^{(T)}(s_T,a_T)\right]$；
* 之后我们应用了动态规划算法：
    $$
    \begin{align}
    V_T^*(s)&=\max_{a_T}R(s_T,a_T)\tag{1}\\
    V_t^*(s)&=\max_a\left(R(s,a)+\sum_{s'}P_{sa}^{(t)}\left(s'\right)V_{t+1}^*\left(s'\right)\right)\tag{2}\\
    \pi_t^*(s)&=\arg\max_a\left(R(s,a)+\sum_{s'}P_{sa}^{(t)}\left(s'\right)V_{t+1}^*\left(s'\right)\right)\tag{3}
    \end{align}
    $$
    其中$(1)$式是有限时域MDP的最后一步；得出最后一步后就可以使用$(2)$式向后递推每一个$V_{T-1},\cdots,V_{0}$；最后，在使用$\arg\max$对每一步价值函数最大化的部分做运算，就可以得到每一步的最优策略。
* 接下来我们介绍了一个具体的LQR问题：
    * 状态为$S\in\mathbb R^n$；
    * 动作为$A\in\mathbb R^d$；
    * 将下一步作为关于当前状态及动作的函数$s_{t+1}=A_ts_t+B_ta_t+w_t$，此处的$w_t\sim\mathcal N(0,\varSigma_w)$是一个期望为零、协方差为$\varSigma_w$的高斯噪音项。
* 上面定义中有了关于当前状态及动作的函数$s_{t+1}=f(s_t,a_t)$，我们就可以选择一个系统长时间保持的状态（系统通常都会都处于这种状态）$(\bar s_t,\bar a_t)$，在该点使用线性函数近似非线性函数$s_{t+1}=f(s_t,a_t)$，进而得到$\displaystyle s_{t+1}\approx f(\bar s_t,\bar a_t)+(\nabla_sf(\bar s_t,\bar a_t))^T(s_t-\bar s_t)+(\nabla_af(\bar s_t,\bar a_t))^T(a_t-\bar a_t)$。最终能够在$(\bar s_t,\bar a_t)$附近得到一个线性函数$s_{t+1}=A_ts_t+B_ta_t$。
* LQR的奖励函数为$\displaystyle R^{(t)}(s_t,a_t)=-\left(s_t^TU_ts_t+a^TV_ta_t\right)$，其中矩阵$U,V$是半正定的，所以这个奖励函数总是非正数；
* 对LQR使用上面的动态规划得到求解方法：
    * 使用$\varPhi_t=-U_t,\varPsi_t=0$初始化$V_T^*$；
    * 向后递推计算$\varPhi_t,\varPsi_t$，使用离散时间Riccati方程，将其当做关于$\varPhi_{t+1},\varPsi_{t+1}$的线性函数；（依次求出$t=T-1,T-2,\cdots,0$。）
        $$
        \begin{align}
        \varPhi_t&=A_t^T\left(\varPhi_{t+1}-\varPhi_{t+1}B_t\left(B_t^T\varPhi_{t+1}B_t-V_t\right)^{-1}B_t\varPhi_{t+1}\right)A_t-U_t\\
        \varPsi_t&=-\mathrm{tr}\varSigma_w\varPhi_{t+1}+\varPsi_{t+1}
        \end{align}
        $$
    * 计算$\displaystyle L_t=\left(B_t^T\varPhi_{t+1}B_t-V_t\right)^{-1}B_t^T\varPhi_{t+1}A_t$，这是一个关于$\varPhi_{t+1},\varPsi_{t+1}$的函数；
    * 计算$\pi^*(s_t)=L_ts_t$，这是一个关于$s_t$的线性函数，系数是$L_t$。
    
    这个算法中有趣的一点是，$L_t$不依靠$\varPsi_t$，$\varPhi_t$也不依赖于$\varPsi_t$。所以，即使我们从不考虑$\varPsi$也不会影响最终的结果。另一个有趣的地方在于$\varSigma_w$只在$\varPsi_t$中出现，再看LQR定义中的$s_{t+1}=A_ts_t+B_ta_t+w_t$可以得出，即使在不知道噪音项协方差的前提下，也可以正确的求出最优策略（但不要忘了最优价值函数是受$\varPsi$影响的）。最后，这两点都是对这个特殊LQR模型（非线性动力系统）独有的，一旦我们对模型做出改变，则这两个特性将不复存在，也就是只有在这个例子中才有“最优策略不受噪音项协方差影响”。在后面讨论Kalman滤波的时候回用到这个性质。

## 微分动态规划（DDP: Differential Dynamic Programming）

## Kalman滤波

## 线性二次型高斯（LQG: Linear quadratic Gaussian）